## General example
This example covers some of the core functionalities of the toolbox.
### Outline
- Load a test dataset
- Preliminary analysis
 - Show summary
 - Show some plots
- Preprocess data
- Evaluate a classifier
- Show some model selection plots

In [ ]:
import sys
#sys.path.append("C:\\Users\\aartetxe\\ARKAITZ\\tesis\\da-toolbox\\datb")
#sys.path.append("C:\\Users\\aartetxe\\ARKAITZ\\tesis\\da-toolbox")

In [ ]:
# Dependencies


In [ ]:
# Allow hot reload of modules
%load_ext autoreload
%autoreload 2
# Show plots within notebook
%matplotlib inline